> **tomato juice dataset**
<br>` 'quality' is the target feature for classification `
<br>` the other features are chemical properties of our product `

**Import the main libraries**

In [ ]:
import numpy as np
import pandas as pd

import warnings
# supress all
warnings.filterwarnings("ignore")

**Import the Dataset**

In [ ]:
## file path: windows style
df = pd.read_csv('..\\datasets\\tomatjus.csv')

## file path: unix style
#df = pd.read_csv('../datasets/tomatjus.csv')

# shape method gives the dimensions of the dataset
print('Dataset dimensions: {} rows, {} columns'.format(
    df.shape[0], df.shape[1]))

In [ ]:
df.info()

***
**Data Preparation and EDA** (unique to this dataset)
* _Check for missing values_
* _Quick visual check of unique values_
* _Split the classification feature out of the dataset_
* _Check column names of categorical attributes ( for get_dummies() )_
* _Check column names of numeric attributes ( for Scaling )_

**Check for missing values**

In [ ]:
cnt=0
print('Missing Values - ')
for col in df.columns:
    nnul = pd.notnull(df[col]) 
    if (len(nnul)!=len(df)):
        cnt=cnt+1
        print('\t',col,':',(len(df)-len(nnul)),'null values')
print('Total',cnt,'features with null values')

# address missing values here

**Quick visual check of unique values, deal with unique identifiers**

In [ ]:
# Identify columns with only one value 
# or with number of unique values == number of rows
n_eq_one = []
n_eq_all = []

print('Unique value count (',df.shape[0],'Rows in the dataset )')
for col in df.columns:
    lc = len(df[col].unique())
    print(col, ' ::> ', lc)
    if lc == 1:
        n_eq_one.append(df[col].name)
    if lc == df.shape[0]:
        n_eq_all.append(df[col].name)

In [ ]:
# Drop columns with only one value
if len(n_eq_one) > 0:
    print('Dropping single-valued features')
    print(n_eq_one)
    data.drop(n_eq_one, axis=1, inplace=True)

# Drop or bin columns with number of unique values == number of rows
if len(n_eq_all) > 0:
    print('Dropping unique identifiers')
    print(n_eq_all)
    data.drop(n_eq_all, axis=1, inplace=True)

# continue with featue selection / feature engineering

**Classification target feature**
<br>"the Right Answers", or more formally "the desired outcome"
<br>Must be in a separate dataset for classification ,,,

_Make it a multi-class problem, using text labels_

In [ ]:
##  divide into classes by giving a range for quality
##  Make it a multi-class problem: {3,4,5} {6} {7.8}
bins = (2, 5, 6, 8)
group_names = ['Average', 'Premium', 'Special']
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names)

* Split the classification feature out of the dataset 

In [ ]:
## Feature being predicted ("the Right Answer")
labels_col = 'quality'
y = df[labels_col]

## Features used for prediction 
# pandas has a lot of rules about returning a 'view' vs. a copy from slice
# so we force it to create a new dataframe 
X = df.copy()
X.drop(labels_col, axis=1, inplace=True)

In [ ]:
# generate a sorted list of unique labels to use later
from sklearn.utils.multiclass import unique_labels
targetlabels = unique_labels(y)

**Check column names of categorical attributes**
<br>Features with text values (categorical attributes) need to be normalised
<br>by changing them to numeric types that the algorithms find easier to work with

In [ ]:
categori = X.select_dtypes(include=['object','category']).columns
print(categori.to_list())

**Check column names of numeric attributes**
<br>Features with numeric values need to be normalised
<br>by changing them to small numbers in a specific range (scaling)

In [ ]:
numeri = X.select_dtypes(include=['float64','int64']).columns
print(numeri.to_list())

In [ ]:
# The proper place to do scaling comes later in the pipeline ,,, 

***
**<br>Create Test // Train Datasets**
> Split X and y datasets into Train and Test subsets,<br>keeping relative proportions of each class (stratify)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=50, stratify=y)

**<br>Target Label Distributions**

In [ ]:
# shape method gives the dimensions of the dataset
print('X_train: {} rows, {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('X_test:  {} rows, {} columns'.format(X_test.shape[0], X_test.shape[1]))
print()
print('y_train: {} rows, 1 column'.format(y_train.shape[0]))
print('y_test:  {} rows, 1 column'.format(y_test.shape[0]))
print()

## Here's a nice report:  
# 1. series to dataframe conversion
my_train = pd.DataFrame(y_train)
my_test = pd.DataFrame(y_test)
# 2. dataframe copy with [[ -- ]]
av_train = my_train[[labels_col]].apply(lambda x: x.value_counts())
av_test = my_test[[labels_col]].apply(lambda x: x.value_counts())
# 3. add a new column
av_train['pct_train'] = round((100 * av_train / av_train.sum()),2)
av_test['pct_test'] = round((100 * av_test / av_test.sum()),2)
# 4. combine the dataframes
av_tt = pd.concat([av_train,av_test], axis=1) 
# 5. print the report
print('Frequency and Distribution of labels')
print(av_tt)

* Class Balance

In [ ]:
from yellowbrick.target import ClassBalance
# The ClassBalance visualizer has a “compare” mode, 
#   to create a side-by-side bar chart instead of a single bar chart 

# Instantiate the visualizer
visualizer = ClassBalance()
visualizer.fit(y_train, y_test)        # Fit the data to the visualizer
_ = visualizer.show()                  # Finalize and render the figure
# assign visualizer.show() to a null variable to avoid printing some trash

***
**<br>Handling Imbalance with Over-sampling // Under-sampling**
<br>_use only the training data and labels!_
<br><br>
Sampling with replacement allows duplicate values, so we only do this for the training data, to give it more to work with. The _pandas dataframe.groupby.sample_ method does not allow sample_amounts to be larger than the group size if replace is False, but if replace is True then replacement will occur even in groups that could have been downsampled.
<br><br>
Here we create a dictionary that maps each class to number of samples, then _groupby.apply_ with a lambda to create the sample, and conditional logic to determine with or without replacement:

In [ ]:
# labels_col = 'quality'
yt = pd.DataFrame(y_train)    # series to dataframe
ff = yt[[labels_col]].apply(lambda x: x.value_counts())
ss = ff[labels_col].to_dict()
ss

In [ ]:
# set new values - anything goes!
ss['Premium'] = 450
ss['Average'] = ss['Average'] - ss['Special']
ss['Special'] = round(ss['Special'] * 2.5)
ss

In [ ]:
# add the labels back to the features dataframe 
xy_train = X_train.copy()
xy_train[labels_col] = yt[labels_col]

In [ ]:
# notice the index numbers - random order from test_train_split()
xy_train.head(4)

In [ ]:
# technically, this is a "one-liner" ...
balanced_df = xy_train.groupby(labels_col, 
                               as_index=False, group_keys=False, sort=False
                        ).apply(lambda g: g.sample(n=ss[g.name],
                                                   replace=(len(g) < ss[g.name])
                                                  )).reset_index(drop=True)

In [ ]:
# we reset the index at the end, so it is neat
balanced_df.head(4)

In [ ]:
## Feature being predicted ("the Right Answer")
ytrain_b = balanced_df[labels_col]

## Features used for prediction 
Xtrain_b = balanced_df.drop(labels_col, axis=1)

In [ ]:
XtrainOriginal = X_train
ytrainOriginal = y_train

X_train = Xtrain_b
y_train = ytrain_b

***
**_This is copied in from above_**

**<br>Target Label Distributions**

In [ ]:
# shape method gives the dimensions of the dataset
print('X_train: {} rows, {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('X_test:  {} rows, {} columns'.format(X_test.shape[0], X_test.shape[1]))
print()
print('y_train: {} rows, 1 column'.format(y_train.shape[0]))
print('y_test:  {} rows, 1 column'.format(y_test.shape[0]))
print()

## Here's a nice report:  
# 1. series to dataframe conversion
my_train = pd.DataFrame(y_train)
my_test = pd.DataFrame(y_test)
# 2. dataframe copy with [[ -- ]]
av_train = my_train[[labels_col]].apply(lambda x: x.value_counts())
av_test = my_test[[labels_col]].apply(lambda x: x.value_counts())
# 3. add a new column
av_train['pct_train'] = round((100 * av_train / av_train.sum()),2)
av_test['pct_test'] = round((100 * av_test / av_test.sum()),2)
# 4. combine the dataframes
av_tt = pd.concat([av_train,av_test], axis=1) 
# 5. print the report
print('Frequency and Distribution of labels')
print(av_tt)

* Class Balance

In [ ]:
from yellowbrick.target import ClassBalance
# The ClassBalance visualizer has a “compare” mode, 
#   to create a side-by-side bar chart instead of a single bar chart 

# Instantiate the visualizer
visualizer = ClassBalance()
visualizer.fit(y_train, y_test)        # Fit the data to the visualizer
_ = visualizer.show()                  # Finalize and render the figure
# assign visualizer.show() to a null variable to avoid printing some trash

***
**_Copy these standard blocks in from another notebook_**

**<br>Function** to calculate perfomance metrics

**<br>Classifier Selection**

**<br>Fit and Predict**

***
**Handling Imbalance with Class Weights**

Balanced weighting a widely used method for imbalanced classification models. It involves applying specified class weights for the majority and minority classes that are used in the classifier training process to achieve better model results.

Unlike over- or under-sampling (a pre-processing step), balanced weighting does not modify the dataset. Instead, each observation is weighted so that wrong predictions for the minority class are given more weight when the loss value is calculated during the model training process. Weights for the loss function can be arbitrary, but a typical choice is class weights (distribution of labels). 

**SKlearn Classifiers with Class Weights**
<br>A limited number of classifiers can take _class_weight='balanced'_ as an argument. This uses the values of y (labels) to automatically adjust weights inversely proportional to class frequencies in the input data (X) as<br>_n_samples / (n_classes * np.bincount(y))_

In [ ]:
# prepare list - these support a class_weight argument
models = []

##  --  Linear  --  ## 
#from sklearn.linear_model import LogisticRegression 
#models.append (("LogReg",LogisticRegression(class_weight='balanced'))) 
#from sklearn.linear_model import SGDClassifier 
#models.append (("StocGradDes",SGDClassifier(class_weight='balanced'))) 

##  --  Support Vector  --  ## 
#from sklearn.svm import SVC 
#models.append(("SupportVectorClf", SVC(class_weight='balanced'))) 
#from sklearn.svm import LinearSVC 
#models.append(("LinearSVC", LinearSVC(class_weight='balanced'))) 
#from sklearn.linear_model import RidgeClassifier
#models.append (("RidgeClf",RidgeClassifier(class_weight='balanced'))) 

##  --  Non-linear  --  ## 
from sklearn.tree import DecisionTreeClassifier 
models.append (("DecisionTree",DecisionTreeClassifier(class_weight='balanced'))) 

##  --  Ensemble: bagging  --  ## 
#from sklearn.ensemble import RandomForestClassifier 
#models.append(("RandomForest", RandomForestClassifier(class_weight='balanced'))) 

print(models)

**_Make sure we are using the right dataset !!_**

In [ ]:
X_train = XtrainOriginal
y_train = ytrainOriginal

***
**_Copy the standard block from above_**

**<br>Fit and Predict**